In [4]:
import pandas as pd
import numpy as np

def process_large_csv(input_file, output_file, kamus_dict, chunk_size=50000):
    """Process large CSV file in chunks with proper data type handling"""
    
    first_chunk = True
    processed_rows = 0
    filtered_rows = 0
    
    for chunk in pd.read_csv(input_file, chunksize=chunk_size, dtype=str, keep_default_na=False, na_filter=False):
        original_chunk_size = len(chunk)
        processed_rows += original_chunk_size
        
        print(f"Processing chunk: {processed_rows} rows...")
        
        # 1. Add Category column based on dictionary
        chunk['Category'] = chunk['brandName'].map(kamus_dict)
        chunk['Category'] = chunk['Category'].fillna('Unknown')
        
        # 2. Convert data types carefully - preserve all original data
        chunk = convert_dtypes_preserve(chunk)
        
        # 3. Filter data - only keep rows with active_days_in_month > 22
        if 'active_days_in_month' in chunk.columns:
            before_filter = len(chunk)
            chunk = chunk[chunk['active_days_in_month'] > 22]
            after_filter = len(chunk)
            filtered_rows += (before_filter - after_filter)
            print(f"  Filtered {before_filter - after_filter} rows (active_days <= 22), keeping {after_filter} rows")
        
        # 4. Save to output file
        if first_chunk:
            chunk.to_csv(output_file, index=False)
            first_chunk = False
        else:
            chunk.to_csv(output_file, mode='a', header=False, index=False)
    
    print(f"Total rows processed: {processed_rows}")
    print(f"Total rows filtered out (active_days <= 22): {filtered_rows}")
    print(f"Final dataset size: {processed_rows - filtered_rows}")

def convert_dtypes_preserve(df):
    """Convert data types while preserving all original data"""
    
    # Store original string values for date columns before conversion
    if 'sales_year' in df.columns:
        df['sales_year_str'] = df['sales_year'].copy()
    if 'sales_month' in df.columns:
        df['sales_month_str'] = df['sales_month'].copy()
    
    # Convert ID columns to int64 - handle NaN by filling with 0
    id_columns = ['menuID', 'branchID', 'branchCompanyID', 'companyID']
    for col in id_columns:
        if col in df.columns:
            # Convert to numeric, coerce errors to NaN, then fill NaN with 0
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
    
    # Convert coordinate columns to float - preserve original null pattern
    coord_columns = ['latitude', 'longitude']
    for col in coord_columns:
        if col in df.columns:
            # Convert to numeric, NaN values will remain as NaN (float)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Convert quantity and monetary columns to appropriate types
    # For active_days_in_month and total_days_in_month, convert to int
    days_columns = ['active_days_in_month', 'total_days_in_month']
    for col in days_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
    
    # For percentage and monetary values, convert to float
    float_columns = [
        'total_qty_monthly', 'gtv_monthly', 'aov_monthly', 'active_days_percentage'
    ]
    
    for col in float_columns:
        if col in df.columns:
            # Clean the data - remove any non-numeric characters except decimal and minus
            cleaned = df[col].str.replace(r'[^\d.-]', '', regex=True)
            # Replace empty strings with 0
            cleaned = cleaned.replace('', '0')
            # Convert to float
            df[col] = pd.to_numeric(cleaned, errors='coerce').fillna(0)
    
    # Convert date columns to numeric but keep string versions
    date_columns = ['sales_year', 'sales_month']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
    
    return df

def create_final_dataframe_with_correct_dtypes():
    """Create final dataframe with exact dtypes matching the trial data"""
    
    # Read the processed data with specific dtypes for date columns
    dtype_spec = {
        'sales_year_str': 'object',
        'sales_month_str': 'object'
    }
    
    try:
        df = pd.read_csv('Data_Final_2024_With_Categories_Intermediate.csv', 
                        dtype=dtype_spec)
    except:
        # If the string columns don't exist, read normally and handle dates differently
        df = pd.read_csv('Data_Final_2024_With_Categories_Intermediate.csv')
    
    # Convert to match trial data types exactly
    df = df.rename(columns={
        'total_qty_monthly': 'total_qty',
        'gtv_monthly': 'gtv_2024', 
        'aov_monthly': 'aov_2024'
    })
    
    # Create SalesDate column from sales_year and sales_month
    # Use string versions if available, otherwise convert numeric to string
    if 'sales_year_str' in df.columns and 'sales_month_str' in df.columns:
        # Use preserved string versions
        df['SalesDate'] = df['sales_year_str'] + '-' + df['sales_month_str'].str.zfill(2) + '-01'
    elif 'sales_year' in df.columns and 'sales_month' in df.columns:
        # Convert numeric columns to string
        df['SalesDate'] = df['sales_year'].astype(str) + '-' + df['sales_month'].astype(str).str.zfill(2) + '-01'
    else:
        df['SalesDate'] = '2024-01-01'  # Default value
    
    # Drop temporary string columns if they exist
    df = df.drop(columns=['sales_year_str', 'sales_month_str'], errors='ignore')
    
    # Select and reorder columns to match trial structure
    trial_columns = [
        'menuID', 'SalesDate', 'cityName', 'brandName', 'branchID', 'branchName',
        'branchCode', 'branchCompanyID', 'latitude', 'longitude', 'subdistrictName',
        'companyID', 'companyName', 'companyCode', 'total_qty', 'gtv_2024', 'aov_2024', 'Category'
    ]
    
    # Only include columns that exist in our dataframe
    available_columns = [col for col in trial_columns if col in df.columns]
    df_final = df[available_columns].copy()
    
    # Ensure final data types match trial exactly
    dtype_mapping = {
        'menuID': 'int64',
        'SalesDate': 'object',
        'cityName': 'object', 
        'brandName': 'object',
        'branchID': 'int64',
        'branchName': 'object',
        'branchCode': 'object',
        'branchCompanyID': 'int64', 
        'latitude': 'float64',
        'longitude': 'float64',
        'subdistrictName': 'object',
        'companyID': 'int64',
        'companyName': 'object',
        'companyCode': 'object',
        'total_qty': 'float64',
        'gtv_2024': 'float64',
        'aov_2024': 'float64',
        'Category': 'object'
    }
    
    for col, dtype in dtype_mapping.items():
        if col in df_final.columns:
            if dtype == 'int64':
                df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0).astype('int64')
            elif dtype == 'float64':
                df_final[col] = pd.to_numeric(df_final[col], errors='coerce').astype('float64')
            else:
                df_final[col] = df_final[col].astype('object')
    
    return df_final

def verify_data_consistency(original_file, processed_file):
    """Verify that no data was lost during processing"""
    
    # Read first few rows of original and processed files
    original_df = pd.read_csv(original_file, nrows=5, dtype=str)
    processed_df = pd.read_csv(processed_file, nrows=5)
    
    print("\n=== DATA CONSISTENCY CHECK ===")
    print(f"Original data shape: {original_df.shape}")
    print(f"Processed data shape: {processed_df.shape}")
    
    # Check if all original rows are preserved
    original_count = sum(1 for _ in open(original_file)) - 1  # exclude header
    processed_count = sum(1 for _ in open(processed_file)) - 1
    
    print(f"Original row count: {original_count}")
    print(f"Processed row count: {processed_count}")
    print(f"Rows preserved: {original_count == processed_count}")
    
    return original_count == processed_count

def main():
    # 1. Read category dictionary from Excel
    print("Reading category dictionary...")
    kamus_df = pd.read_excel('Kamus.xlsx')
    kamus_dict = dict(zip(kamus_df['Brand Name'], kamus_df['Category']))
    print(f"Loaded {len(kamus_dict)} brand-category mappings")
    
    # 2. Process the large CSV file
    input_csv = 'Data_Final_2024.csv'
    intermediate_output = 'Data_Final_2024_With_Categories_Intermediate.csv'
    final_output = 'Data_Final_2024_With_Categories_Final.csv'
    
    print("\nStarting categorization process...")
    process_large_csv(input_csv, intermediate_output, kamus_dict)
    
    # 3. Create final dataframe with correct structure
    print("\nCreating final dataframe structure...")
    final_df = create_final_dataframe_with_correct_dtypes()
    
    # 4. Save final result
    final_df.to_csv(final_output, index=False)
    
    # 5. Verification
    print("\n=== FINAL VERIFICATION ===")
    consistency_check = verify_data_consistency(input_csv, final_output)
    
    if consistency_check:
        print("\n✅ SUCCESS: All data preserved during processing!")
    else:
        print("\n⚠️ WARNING: Some data may have been lost during processing!")
    
    # 6. Show final data info
    print(f"\nFinal dataframe shape: {final_df.shape}")
    print("\nData types:")
    print(final_df.dtypes)
    
    print("\nNon-null counts:")
    for col in final_df.columns:
        non_null = final_df[col].notna().sum()
        total = len(final_df)
        print(f"{col}: {non_null}/{total} ({(non_null/total*100):.2f}%)")
    
    # Check active_days_in_month distribution in final data
    if 'active_days_in_month' in final_df.columns:
        active_days_stats = final_df['active_days_in_month'].describe()
        print(f"\n📊 Active Days in Month Statistics (After Filtering > 22 days):")
        print(f"   - Min: {active_days_stats['min']}")
        print(f"   - Max: {active_days_stats['max']}")
        print(f"   - Mean: {active_days_stats['mean']:.2f}")
        print(f"   - Median: {final_df['active_days_in_month'].median()}")
    
    # Compare with original
    original_df = pd.read_csv(input_csv, dtype=str, nrows=5)
    print(f"\nOriginal had {len(original_df.columns)} columns, final has {len(final_df.columns)} columns")
    
    print("\nSample of categorized data:")
    print(final_df[['brandName', 'Category', 'active_days_in_month']].head(10))

# Alternative: Direct processing without intermediate file
def process_directly_final():
    """Process the entire file at once with correct final structure"""
    
    # Read category dictionary
    kamus_df = pd.read_excel('Kamus.xlsx')
    kamus_dict = dict(zip(kamus_df['Brand Name'], kamus_df['Category']))
    
    # Read the CSV with all columns as string initially
    print("Reading CSV file...")
    df = pd.read_csv('Data_Final_2024.csv', dtype=str, keep_default_na=False, na_filter=False)
    
    print(f"Original data shape: {df.shape}")
    
    # Add category column
    df['Category'] = df['brandName'].map(kamus_dict)
    df['Category'] = df['Category'].fillna('Unknown')
    
    # Convert data types
    df = convert_dtypes_for_final(df)
    
    # Filter data - only keep rows with active_days_in_month > 22
    if 'active_days_in_month' in df.columns:
        before_filter = len(df)
        df = df[df['active_days_in_month'] > 22]
        after_filter = len(df)
        print(f"Filtered {before_filter - after_filter} rows (active_days <= 22), keeping {after_filter} rows")
    
    # Create SalesDate column (using string columns directly)
    if 'sales_year' in df.columns and 'sales_month' in df.columns:
        df['SalesDate'] = df['sales_year'] + '-' + df['sales_month'].str.zfill(2) + '-01'
    else:
        df['SalesDate'] = '2024-01-01'
    
    # Rename columns to match trial structure
    df = df.rename(columns={
        'total_qty_monthly': 'total_qty',
        'gtv_monthly': 'gtv_2024', 
        'aov_monthly': 'aov_2024'
    })
    
    # Select final columns
    trial_columns = [
        'menuID', 'SalesDate', 'cityName', 'brandName', 'branchID', 'branchName',
        'branchCode', 'branchCompanyID', 'latitude', 'longitude', 'subdistrictName',
        'companyID', 'companyName', 'companyCode', 'total_qty', 'gtv_2024', 'aov_2024', 'Category'
    ]
    
    available_columns = [col for col in trial_columns if col in df.columns]
    df_final = df[available_columns].copy()
    
    # Convert to final dtypes
    df_final = convert_to_final_dtypes(df_final)
    
    # Save result
    output_file = 'Data_Final_2024_With_Categories_Final.csv'
    df_final.to_csv(output_file, index=False)
    
    print(f"Processed data shape: {df_final.shape}")
    print(f"Data saved to: {output_file}")
    
    return df_final

def convert_dtypes_for_final(df):
    """Convert data types for direct processing approach"""
    
    # Convert ID columns to int64
    id_columns = ['menuID', 'branchID', 'branchCompanyID', 'companyID']
    for col in id_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
    
    # Convert coordinate columns to float
    coord_columns = ['latitude', 'longitude']
    for col in coord_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert days columns to int
    days_columns = ['active_days_in_month', 'total_days_in_month']
    for col in days_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
    
    # Convert monetary columns to float
    float_columns = [
        'total_qty_monthly', 'gtv_monthly', 'aov_monthly', 'active_days_percentage'
    ]
    
    for col in float_columns:
        if col in df.columns:
            cleaned = df[col].str.replace(r'[^\d.-]', '', regex=True).replace('', '0')
            df[col] = pd.to_numeric(cleaned, errors='coerce').fillna(0)
    
    return df

def convert_to_final_dtypes(df):
    """Convert to final dtypes matching trial structure"""
    
    dtype_mapping = {
        'menuID': 'int64',
        'SalesDate': 'object',
        'cityName': 'object', 
        'brandName': 'object',
        'branchID': 'int64',
        'branchName': 'object',
        'branchCode': 'object',
        'branchCompanyID': 'int64', 
        'latitude': 'float64',
        'longitude': 'float64',
        'subdistrictName': 'object',
        'companyID': 'int64',
        'companyName': 'object',
        'companyCode': 'object',
        'total_qty': 'float64',
        'gtv_2024': 'float64',
        'aov_2024': 'float64',
        'Category': 'object'
    }
    
    for col, dtype in dtype_mapping.items():
        if col in df.columns:
            if dtype == 'int64':
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
            elif dtype == 'float64':
                df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')
    
    return df

if __name__ == "__main__":
    # Choose one method:
    
    # Method 1: Chunk processing (recommended for very large files)
    # main()
    
    # Method 2: Direct processing (simpler, less error-prone)
    df = process_directly_final()

Reading CSV file...
Original data shape: (1587824, 21)
Filtered 1276415 rows (active_days <= 22), keeping 311409 rows
Processed data shape: (311409, 18)
Data saved to: Data_Final_2024_With_Categories_Final.csv
